In [1]:
import jieba
#构建中文分词类v3.0
class Leijieba:    
    
    __stopwords = ''
        
    def __init__(self,s,t='',b=''):    #t为停用词的文件参数（路径+文件名）；b为本地词库的文件参数（路径+文件名）
        self.__s = s
        
        if t != '':
            self.__stopwords = [line.strip() for line in open(t,encoding='UTF-8').readlines()]  #逐行读取停用词文件，放入类内部变量__stopwords
            
        if b != '':
            jieba.load_userdict(b)
        
    def cut(self):
        seg_list = jieba.cut(self.__s, cut_all=False)
        if self.__stopwords != '':
            outstr = []    #目的是存储去掉停用用词的分词结果
            for word in list(seg_list):
                if word not in self.__stopwords:
                    if word.strip() !='':   #去空格
                        outstr.append(word)
                        
            seg_list = outstr                                                
        return list(seg_list)
    
    def num(self):
        lst = self.cut()
        
        word_dict = {}        
        for item in lst:
            if item not in word_dict:
                word_dict[item] = 1   #新建“键-值”
            else:
                word_dict[item] += 1  #增加 键的值
        return sorted(word_dict.items(), key=lambda e:e[1], reverse=True)  
    
    def cut1(self,num = 0,sw = 0):   #1.删除词频低于lnum的词,2.if sw ==1,删除单字的词
        lst = self.cut()
        
        word_dict = {}        
        for item in lst:
            if item not in word_dict:
                word_dict[item] = 1   #新建“键-值”
            else:
                word_dict[item] += 1  #增加 键的值
        dic = dict(sorted(word_dict.items(), key=lambda e:e[1], reverse=True)) 
        lst1 = lst[:]
        
        if sw == 1:
            for item in lst:
                if len(item) == 1:
                    lst1.remove(item)
                else:
                    if dic[item] <= num:
                        lst1.remove(item)
        else:
            for item in lst:
                if dic[item] <= num:
                    lst1.remove(item)
                
        return lst1 

In [2]:
import pandas as pd
txtfile = pd.read_csv('cnews.txt', sep='\t', encoding = 'utf-8', header = None)

In [3]:
import re
for i in range(len(txtfile)):
    txtfile[1][i] = re.sub(r'[^\u4e00-\u9fa5]+',' ',txtfile[1][i])

In [4]:
s = 'AAA我是分词的间隔符BBB'.join(list(txtfile[1]))
m = Leijieba(s,t='stopwords.txt',b='userdic.txt')
s1 = ' '.join(m.cut1(10,1))
txtf = open('cnews.jieba.txt','wt', encoding='utf-8')
txtf.write(s1.replace('AAA我是分词的间隔符BBB','\n'))
txtf.close()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lb\AppData\Local\Temp\jieba.cache
Loading model cost 0.620 seconds.
Prefix dict has been built successfully.


In [5]:
txt = pd.read_csv('cnews.jieba.txt', sep='\t', encoding = 'utf-8', header = None)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', max_features=4000,  lowercase = False)
vectorizer.fit(list(txt[0]))
X = vectorizer.transform(list(txt[0]))

In [7]:
import numpy as np
ordering = ['体育','娱乐','家居','房产','教育','时尚','时政','游戏','科技','财经']
y = np.array(txtfile[0].map(lambda x: ordering.index(x)))

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [9]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

In [10]:
y_pred = classifier.predict(X_test)

In [11]:
from sklearn.metrics import classification_report
t = classification_report(y_test, y_pred, target_names = ordering)
print(t)

              precision    recall  f1-score   support

          体育       1.00      0.98      0.99       124
          娱乐       0.94      0.98      0.96       121
          家居       0.97      0.90      0.93       127
          房产       0.99      0.93      0.96       150
          教育       0.97      1.00      0.99       108
          时尚       0.93      0.98      0.95       129
          时政       0.95      0.92      0.94       111
          游戏       1.00      0.99      1.00       130
          科技       0.99      1.00      1.00       128
          财经       0.95      1.00      0.97       122

    accuracy                           0.97      1250
   macro avg       0.97      0.97      0.97      1250
weighted avg       0.97      0.97      0.97      1250

